# TESTTESTTESTTESTTESTTESTTESTTESTTESTTEST

In [ ]:
%reload_ext autoreload
%autoreload 2
from hw5_funct import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from numba import jit, autojit
import multiprocessing
from itertools import repeat

X_data, Y_data, activities = get_data('./data/')

In [ ]:
@autojit
def get_accuracy_hierarchical(window, K_0, K_1, X_data, Y_data):
    rfscores = []
    for i in range(10):
        X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, stratify=Y_data)
        vq_dict = get_vq_dict_hierarchical(X_train, window, K_0, K_1, 100)
        X_train_VQ = vec_quantize_signals(X_train, vq_dict)
        X_test_VQ = vec_quantize_signals(X_test, vq_dict)
        rf = RandomForestClassifier(n_estimators=100, max_depth=50)
        rf.fit(X_train_VQ, Y_train)
        rfscore = rf.score(X_test_VQ,Y_test)
        rfscores.append(rfscore)
    return np.mean(rfscores)

In [ ]:
%%time
windows = [3*9]
kvs = [(40, 12), (12, 40), (20, 25), (25, 20)]
for window in windows:
    for (K_0, K_1) in kvs:
        rfscore = get_accuracy_hierarchical(window, K_0, K_1, X_data, Y_data)
        print('%d, %d, %d, %.5f' % (window, K_0, K_1, rfscore))